<a href="https://colab.research.google.com/github/ssoyeonn/Tobigs20/blob/main/Week7.%20SVM%20%26%20NLP%20Basic%20%26%20Vision%20Basic/Week7_NLPBasic_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Basic Assignment
## 과제 : spam.csv를 활용하여 유의미한 해석을 도출해주세요!

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다.
- 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [112]:
spam = pd.read_csv('/content/drive/MyDrive/Tobigs20_week7_NLPBasic/spam.csv')

In [104]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [105]:
spam

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [106]:
spam.isna().sum().sum()

0

## Tokenizing


In [107]:
import nltk

In [117]:
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

print(word_tokenize(spam.iloc[5]['v2']))
print(WordPunctTokenizer().tokenize(spam.iloc[5]['v2']))
print(text_to_word_sequence(spam.iloc[5]['v2']))

['FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å£1.50', 'to', 'rcv']
['FreeMsg', 'Hey', 'there', 'darling', 'it', "'", 's', 'been', '3', 'week', "'", 's', 'now', 'and', 'no', 'word', 'back', '!', 'I', "'", 'd', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å', '£', '1', '.', '50', 'to', 'rcv']
['freemsg', 'hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back', "i'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'to', 'send', 'å£1', '50', 'to', 'rcv']


- WordPunctTokenizer 는 구두점 또한 별도로 분리함
- text_to_word_sequence는 it's, i'd 등을 하나의 토큰으로 분리하고 모든 문자를 자동으로 소문자로 바꿔줌

여기서는 word_tokenize를 사용하기로 함.

In [118]:
result = []
for phrase in spam['v2'] :
    words = word_tokenize(phrase)
    result.append(words)

spam['v2'] = result

Stopwords Removal

In [128]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

stopword_removed = []
for phrase in spam['v2']:
    words = [w for w in phrase if w not in stop_words]
    stopword_removed.append(words)

spam['v2'] = stopword_removed

print(spam['v2'])

0       [Go, jurong, point, ,, crazy, .., Available, b...
1                [Ok, lar, ..., Joking, wif, u, oni, ...]
2       [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3       [U, dun, say, early, hor, ..., U, c, already, ...
4       [Nah, I, n't, think, goes, usf, ,, lives, arou...
                              ...                        
5567    [This, 2nd, time, tried, 2, contact, u., U, å£...
5568         [Will, Ì_, b, going, esplanade, fr, home, ?]
5569       [Pity, ,, *, mood, ., So, ..., suggestions, ?]
5570    [The, guy, bitching, I, acted, like, 'd, inter...
5571                           [Rofl, ., Its, true, name]
Name: v2, Length: 5572, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram 등이 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

**1) One-hot encoding**

In [120]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 정수 인코딩
labels = spam['v1'].tolist()
texts = spam['v2'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_to_index = tokenizer.word_index
print(word_to_index)

encoded = tokenizer.texts_to_sequences(texts)[0]
print(encoded)

{'.': 1, 'i': 2, ',': 3, '?': 4, '!': 5, 'u': 6, '...': 7, '&': 8, ';': 9, ':': 10, '..': 11, 'call': 12, ')': 13, "'s": 14, '2': 15, "'m": 16, 'get': 17, 'ur': 18, "n't": 19, 'you': 20, 'gt': 21, 'lt': 22, '*': 23, '4': 24, "''": 25, '#': 26, 'go': 27, 'free': 28, 'ok': 29, 'know': 30, 'got': 31, 'like': 32, 'good': 33, "'ll": 34, 'come': 35, '-': 36, 'time': 37, 'day': 38, 'love': 39, 'want': 40, 'send': 41, 'no': 42, 'text': 43, 'one': 44, 'going': 45, 'txt': 46, 'need': 47, 'home': 48, 'lor': 49, 'how': 50, 'r': 51, 'sorry': 52, 'see': 53, 'still': 54, 'back': 55, 'today': 56, 'stop': 57, 'da': 58, 'n': 59, 'do': 60, 'reply': 61, 'if': 62, 'we': 63, 'but': 64, 'mobile': 65, 'dont': 66, 'tell': 67, 'new': 68, 'later': 69, 'take': 70, 'think': 71, 'so': 72, 'hi': 73, 'please': 74, 'just': 75, 'phone': 76, 'what': 77, 'pls': 78, 'my': 79, 'the': 80, 'it': 81, 'ì_': 82, 'dear': 83, '(': 84, 'week': 85, 'well': 86, 'your': 87, 'great': 88, 'night': 89, 'claim': 90, 'k': 91, 'much': 92, 

In [121]:
from tensorflow.keras.utils import to_categorical

one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [122]:
np.array(one_hot).shape

(20, 4433)

**2) Word2Vec**

1) ham 데이터에 대한 워드임베딩

In [135]:
tokenized_ham = spam[spam['v1'] == 'ham']['v2']

model = Word2Vec(tokenized_ham, window=3, sg=0)

model.save('cbow_ham')

cbow_ham = Word2Vec.load('cbow_ham')

similarity = cbow_ham.wv.most_similar(positive=["free"], topn=10)
similarity

[('time', 0.9994563460350037),
 ('buy', 0.9994272589683533),
 ('late', 0.999409019947052),
 ('cos', 0.9993587732315063),
 ("'ve", 0.9993262887001038),
 ('If', 0.9993211627006531),
 ('make', 0.9993206262588501),
 ('see', 0.9993109107017517),
 ('keep', 0.9993019700050354),
 ('Can', 0.9992824196815491)]

In [134]:
tokenized_ham = spam[spam['v1'] == 'ham']['v2']

model = Word2Vec(tokenized_ham, window=3, sg=1)

model.save('skipgram_ham')

skipgram_ham = Word2Vec.load('skipgram_ham')

similarity = skipgram_ham.wv.most_similar(positive=["free"], topn=10)
similarity

[('talk', 0.9940818548202515),
 ('missed', 0.9937893152236938),
 ('yeah', 0.9936849474906921),
 ('Sir', 0.993626058101654),
 ('carlos', 0.9934936165809631),
 ('parents', 0.9933820366859436),
 ('took', 0.9933311939239502),
 ('asked', 0.992902934551239),
 ('bath', 0.9928829073905945),
 ('uncle', 0.9927734136581421)]

2) spam 데이터에 대한 워드임베딩

In [138]:
tokenized_spam = spam[spam['v1'] == 'spam']['v2']

model = Word2Vec(tokenized_spam, window=5, sg=0)

model.save('cbow_spam')

cbow_spam = Word2Vec.load('cbow_spam')

similarity = cbow_spam.wv.most_similar(positive=["free"], topn=10)
similarity

[('?', 0.9996550679206848),
 (',', 0.9996388554573059),
 ('&', 0.9996216297149658),
 ('!', 0.9996210336685181),
 ('.', 0.9995975494384766),
 ('4', 0.9995739459991455),
 ('Nokia', 0.9995484948158264),
 ('FREE', 0.9995480179786682),
 ('ur', 0.9995468854904175),
 ('Call', 0.9995452761650085)]

In [132]:
tokenized_spam = spam[spam['v1'] == 'spam']['v2']

model = Word2Vec(tokenized_spam, window=3, sg=1)

model.save('skipgram_spam')

skipgram_spam = Word2Vec.load('skipgram_spam')

similarity = skipgram_spam.wv.most_similar(positive=["free"], topn=10)
similarity

[('?', 0.998005747795105),
 ('sexy', 0.9979769587516785),
 ('new', 0.9979604482650757),
 ('Reply', 0.9979122877120972),
 ('back', 0.9978794455528259),
 ('A', 0.9978784322738647),
 ('video', 0.9978676438331604),
 ('18', 0.9978640675544739),
 ('please', 0.9978556632995605),
 ('Video', 0.9977919459342957)]

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.

In [141]:
pd.DataFrame(cbow_ham.wv.most_similar('free'))

,0,1
0,time,0.999456
1,buy,0.999427
2,late,0.999409
3,cos,0.999359
4,'ve,0.999326
5,If,0.999321
6,make,0.999321
7,see,0.999311
8,keep,0.999302
9,Can,0.999282


In [136]:
pd.DataFrame(skipgram_ham.wv.most_similar('free'))

,0,1
0,talk,0.994082
1,missed,0.993789
2,yeah,0.993685
3,Sir,0.993626
4,carlos,0.993494
5,parents,0.993382
6,took,0.993331
7,asked,0.992903
8,bath,0.992883
9,uncle,0.992773


- 정상 메세지에서는 사람이 인지하기에는 특별한 상관관계가 보이지 않는 단어들이 free와 유사한 단어들로 검출되었다.

In [140]:
pd.DataFrame(cbow_spam.wv.most_similar('free'))

,0,1
0,?,0.999655
1,",",0.999639
2,&,0.999622
3,!,0.999621
4,.,0.999598
5,4,0.999574
6,Nokia,0.999548
7,FREE,0.999548
8,ur,0.999547
9,Call,0.999545


In [137]:
pd.DataFrame(skipgram_spam.wv.most_similar('free'))

,0,1
0,?,0.998006
1,sexy,0.997977
2,new,0.997960
3,Reply,0.997912
4,back,0.997879
5,A,0.997878
6,video,0.997868
7,18,0.997864
8,please,0.997856
9,Video,0.997792


스팸 메세지에는 휴대폰, 전화, 영상, 답장 등과 관련된 단어가 free와 유사한 단어로 검출되었다.